In [698]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g4RqYr9VFVNXm3EVQ2O8q3tEDEHjdYugslOtIOEogZTwVb9hX8iVTw

Successfully saved authorization token.


In [699]:
from geetools import tools, utils
import geedatasets
import utils
import pandas as pd
import altair as alt
import numpy as np
import geopandas as gpd
import folium
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon
#import numpy
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt

In [744]:
collectionName_L5 = "LANDSAT/LT05/C01/T1_TOA" 

In [700]:
#Tasseled caps

#Calculate the Tasseled cap indices according the following transformations :
#Transformations:
#landsat 5 Crist (1985)
#Landsat 7 Huang et al. (2002)
#landsat 8 Baig et al. (2014)

#Tasseled cap brightness
def tasseled_b_L5(image):
    return ee.Image(0).expression(
           '((B1 * c1) + (B2*c2) + (B3*c3) + (B4*c4) + (B5*c5) + (B7*c7))',
         {
             'B1':image.select('B1'),
             'B2':image.select('B2'),
             'B3':image.select('B3'),
             'B4':image.select('B4'),
             'B5':image.select('B5'),
             'B7':image.select('B7'),
             'c1': ee.Number(0.2043),
             'c2': ee.Number(0.4158),
             'c3': ee.Number(0.5524),
             'c4': ee.Number(0.5741),
             'c5': ee.Number(0.3124),
             'c7': ee.Number(0.2303)
        }).set('system:time_start', image.get('system:time_start')).rename('brightness')


#-----------------------------------------------------------------------------------------------------------------#

#Tasseled cap greenness
def tasseled_g_L5(image):
    return ee.Image(0).expression(
           '((B1 * c1) + (B2*c2) + (B3*c3) + (B4*c4) + (B5*c5) + (B7*c7))',
         {
             'B1':image.select('B1'),
             'B2':image.select('B2'),
             'B3':image.select('B3'),
             'B4':image.select('B4'),
             'B5':image.select('B5'),
             'B7':image.select('B7'),
             'c1': ee.Number(-0.16033),
             'c2': ee.Number(-0.2819),
             'c3': ee.Number(-0.4934),
             'c4': ee.Number(0.7940),
             'c5': ee.Number(-0.0002),
             'c7': ee.Number(-0.1446)
        }).set('system:time_start', image.get('system:time_start')).rename('greenness')


#----------------------------------------------------------------------------------------------------------------#

#Tasseled cap wetness    
def tasseled_w_L5(image):
    return ee.Image(0).expression(
           '((B1 * c1) + (B2*c2) + (B3*c3) + (B4*c4) + (B5*c5) + (B7*c7))',
         {
             'B1':image.select('B1'),
             'B2':image.select('B2'),
             'B3':image.select('B3'),
             'B4':image.select('B4'),
             'B5':image.select('B5'),
             'B7':image.select('B7'),
             'c1': ee.Number(0.0315),
             'c2': ee.Number(0.2021),
             'c3': ee.Number(0.3102),
             'c4': ee.Number(0.1594),
             'c5': ee.Number(-0.6806),
             'c7': ee.Number(-0.6109)
        }).set('system:time_start', image.get('system:time_start')).rename('wetness')
        

#----------------------------------------------------------------------------------------------------------------#
#Mask cloud and Shadow for Landsat images L5 TOA

def cloudMaskL457(image):
    qa = image.select('BQA')
    #If the cloud bit (4) is set and the cloud confidence (6) is high
    #or the cloud shadow bit is set (8), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 4).And(qa.bitwiseAnd(1 << 6)).Or(qa.bitwiseAnd(1 << 8))
    #Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)


def NDWI_mask(image):
    ndwi_mask = image.normalizedDifference(['B2', 'B4']).rename('NDWI').set('system:time_start', image.get('system:time_start'))
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(ndwi_mask.lte(0.3).Not()).updateMask(mask2)


def NDVI_mask(image):
    ndvi_mask = image.normalizedDifference(['B4', 'B3']).rename('NDVI').set('system:time_start', image.get('system:time_start'))
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(ndvi_mask.lt(0).Not()).updateMask(mask2)

# def snowMaskL457(image):
#     qa = image.select('BQA')
#     #If the cloud bit (5) is set and the cloud confidence (7) is high
#     #or the cloud shadow bit is set (3), then it's a bad pixel.
#     cloud = qa.bitwiseAnd(1 << 9).And(qa.bitwiseAnd(1 << 10))
#     #Remove edge pixels that don't occur in all bands
#     mask2 = image.mask().reduce(ee.Reducer.min())
#     return image.updateMask(cloud.Not()).updateMask(mask2)



# def mask_water2(image):
#     #Get the pixel QA band.
#     qa = image.select('BQA')
#     mask = qa.bitwiseAnd(10).neq(0)
#     mask2 = image.mask().reduce(ee.Reducer.min())
#     return image.updateMask(mask).updateMask(mask2)


**Bitmask for BQA**
<br>Bit 0: Designated Fill
<br>0: No
<br>1: Yes
<br>Bit 1: Designated Pixel
<br>0: No
<br>1: Yes
<br>Bits 2-3: Radiometric Saturation
<br>0: No bands contain saturation
<br>1: 1-2 bands contain saturation
<br>2: 3-4 bands contain saturation
<br>3: 5 or more bands contain saturation
<br>Bit 4: Cloud
<br>0: No
<br>1: Yes
<br>Bits 5-6: Cloud Confidence
<br>0: Not Determined / Condition does not exist.
<br>1: Low, (0-33 percent confidence)
<br>2: Medium, (34-66 percent confidence)
<br>3: High, (67-100 percent confidence)
<br>Bits 7-8: Cloud Shadow Confidence
<br>0: Not Determined / Condition does not exist.
<br>1: Low, (0-33 percent confidence)
<br>2: Medium, (34-66 percent confidence)
<br>3: High, (67-100 percent confidence)
<br>Bits 9-10: Snow / Ice Confidence
<br>0: Not Determined / Condition does not exist.
<br>1: Low, (0-33 percent confidence)
<br>2: Medium, (34-66 percent confidence)
<br>3: High, (67-100 percent confidence)

In [746]:
def run_export(image, crs, filename, scale, region, maxPixels=1e13):
    '''
    Runs an export function on GEE servers
    '''
    task_config = {'fileNamePrefix': filename,'crs': crs,'scale': scale,'maxPixels': maxPixels,'fileFormat': 'GeoTIFF','region': region,}
    task = ee.batch.Export.image.toDrive(image, filename, **task_config)
    task.start()

In [747]:
def gee_geometry_from_shapely(geom, crs='epsg:4326'):
    """ 
    Simple helper function to take a shapely geometry and a coordinate system and convert them to a 
    Google Earth Engine Geometry.
    """
    from shapely.geometry import mapping
    ty = geom.type
    if ty == 'Polygon':
        return ee.Geometry.Polygon(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)
    elif ty == 'Point':
        return ee.Geometry.Point(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)    
    elif ty == 'MultiPolygon':
        return ee.Geometry.MultiPolygon(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)

**system:time_start**
<br>The Earth Engine time stamp in milliseconds since the UNIX epoch. See this link (https://en.wikipedia.org/wiki/Unix_time) for more information. The time stamp is set to the nominal image acquisition time for single scenes. It is set to the nominal composite start period for temporal composites.

In [748]:
#Returns the difference between two Dates in the specified units; 
#the result is floating-point and based on the average length of the unit.
#ee.Date.difference(start, unit)
# Arguments:
# this:date (Date)
# start (Date)
# unit (String):
# One of 'year', 'month' 'week', 'day', 'hour', 'minute', or 'second'.
#Returns: Float
    
    
def addDate(image):
        date = ee.Date(image.get('system:time_start')) #
        years = date.difference(ee.Date('1984-01-01'), 'year');
        return image.addBands(ee.Image.constant(years)).addBands(ee.Image(years).rename('t')).float()

In [749]:
#Load Image Collection
collection_L5 = ee.ImageCollection(collectionName_L5)

In [756]:
# Define the area of interest

from shapely.geometry import Polygon
    
geom = Polygon([[-122.7609625539781,71.0248652750595],
                [-121.7502203664781,71.35078807875405],
                [-121.0031500539781,71.37185539607945],
                [-120.3219977102281,71.53956940315283],
                [-120.3000250539781,71.98650637573677],
                [-119.8825445852281,72.18251814893772],
                [-119.2233648977281,72.33650688912165],
                [-119.0695563039781,72.62751123507817],
                [-118.3664313039781,72.77777272011134],
                [-117.4216070852281,73.02327346292203],
                [-116.7624273977281,73.12562314751086],
                [-115.6637945852281,73.35372393902468],
                [-115.1803961477281,73.49164105978065],
                [-117.2238531789781,74.21859485977957],
                [-118.1247320852281,74.30204109643341],
                [-118.8498297414781,74.23054216680322],
                [-119.5749273977281,74.24844660046523],
                [-121.2448492727281,74.56735766699148],
                [-121.7941656789781,74.57904742715868],
                [-122.3654547414781,74.4970373362395],
                [-124.8483648977281,74.36730409994937],
                [-124.6066656789781,74.1226983323726],
                [-124.1672125539781,73.81934897680587],
                [-124.0793219289781,73.57882746350154],
                [-124.4528570852281,73.39772768318414],
                [-124.8263922414781,73.21468747419634],
                [-124.9802008352281,72.96543724049849], 
                [-125.3097906789781,72.81676365632822],
                [-125.1999273977281,72.620949382208],
                [-125.9469977102281,72.08139173142834],
                [-125.8371344289781,71.9116129732407],
                [-125.3976813039781,71.93206824488587],
                [-124.4308844289781,71.71961482925289],
                [-123.9914313039781,71.56737970042492],
                [-123.7057867727281,71.39289975804704],
                [-123.4421148977281,71.06055580588858],
                [-122.7609625539781,71.0248652750595] ])
search_area = gee_geometry_from_shapely(geom)

In [757]:
# Filter the collections
collection_5 = ee.ImageCollection(collection_L5).filter(ee.Filter.date('1984-01-01', '2014-01-01')
     .calendarRange(6,8, 'month'))\
    .filterBounds(search_area)\
    .filter(ee.Filter.lt('CLOUD_COVER', 30))\
    .map(maskCL)\
    .map(NDVI_mask)\
    .sort('system:time_start')

In [758]:
#Tasseled cap
#Call the function that calculate TCB, TCG and TCW for Landsat 5 and add a date band

#TCB
tcb_5=collection_5.map(tasseled_b_L5)
tcb_5_date=tcb_5.map(addDate);

#TCG
tcg_5=collection_5.map(tasseled_g_L5)
tcg_5_date=tcg_5.map(addDate);


#TCW
tcw_5=collection_5.map(tasseled_w_L5)
tcw_5_date=tcw_5.map(addDate);


In [731]:
tcg_5_date.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'LANDSAT/LT05/C01/T1_TOA',
 'version': 1612811827293391,
 'properties': {'type_name': 'ImageCollection',
  'visualization_1_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_TOA_thumb.png',
  'description': '<p>Landsat 5 TM Collection 1 Tier 1\n  calibrated top-of-atmosphere (TOA) reflectance.\n  Calibration coefficients are extracted from the image metadata.  See<a href="http://www.sciencedirect.com/science/article/pii/S0034425709000169">\n  Chander et al. (2009)</a> for details on the TOA computation.</p></p>\n<p><b>Revisit Interval</b>\n<br>\n  16 days\n</p>\n<p><b>Bands</b>\n<table class="eecat">\n<tr>\n<th scope="col">Name</th>\n<th scope="col">Pixel Size</th>\n<th scope="col">Wavelength</th>\n<th scope="col">Description</th>\n</tr>\n<tr>\n<td>B1</td>\n<td>\n      30 meters\n</td>\n<td>0.45 - 0.52 µm</td>\n<td><p>Blue</p></td>\n</tr>\n<tr>\n<td>B2</td>\n<td>\n      30 meters\n</td>\n<td>0.52 - 0.60 µm</td>

**ee.Reducer.linearFit()**
<br>Computes the least squares estimate of a linear function of one variable with a constant term.
<br>The data should be set up as a two-band input image, where the first band is the independent variable (time) and the <br>second band is the dependent variable. 
<br>https://developers.google.com/earth-engine/guides/reducers_regression

In [759]:
# linear fit TCG,TCB,TCW
linear_fit_TCG = tcg_5_date.select(['t', 'greenness']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_TCB = tcb_5_date.select(['t', 'brightness']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_TCW = tcw_5_date.select(['t', 'wetness']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])

In [760]:
#Export the images to google drive

In [761]:
run_export(linear_fit_TCG, 'epsg:4326', 'Lin_TCG_1984_2011_Summer_snow', scale=30, region=search_area, maxPixels=1e13)

In [762]:
run_export(linear_fit_TCB, 'epsg:4326', 'Lin_TCB_1984_2011_Summer_snow', scale=30, region=search_area, maxPixels=1e13)

In [763]:
run_export(linear_fit_TCW, 'epsg:4326', 'Lin_TCW_1984_2011_Summer_snow', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
# print('linear_fit_TCW estimates:', linear_fit_TCW);
# print('y-intercept:', linear_fit_TCW.get('offset'));
# print('Slope:', linear_fit_TCW.get('scale'));

In [113]:
tcw_5_date.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'LANDSAT/LT05/C01/T1_SR',
 'version': 1611826450755576,
 'properties': {'type_name': 'ImageCollection',
  'visualization_1_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png',
  'description': '<p>This dataset is the atmospherically\ncorrected surface reflectance from  the Landsat 5 ETM sensor.\nThese images contain 4 visible and near-infrared (VNIR) bands and 2 short-wave\ninfrared (SWIR) bands processed to orthorectified surface reflectance,\nand one thermal infrared (TIR) band processed to orthorectified brightness\ntemperature.  The VNIR and SWIR bands have a resolution of 30m / pixel.\nThe TIR band, while originally collected with a resolution of 120m / pixel\n(60m / pixel for Landsat 7) has been resampled using cubic convolution to 30m.</p><p>These data have been atmospherically corrected using\n<a href="https://www.usgs.gov/media/files/landsat-4-7-surface-reflectance-code-ledaps-product-guide